In [ ]:
from pathlib import Path
import sys

import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
sys.path.append('..')
from networks.osvos_resnet import OSVOS_RESNET

In [ ]:
def get_net() -> nn.Module:
    net = OSVOS_RESNET(pretrained=False)
    net = net.cuda()
    path_model = Path('../models/resnet18_11_11_blackswan_epoch-9999.pth')
    parameters = torch.load(str(path_model), map_location=lambda storage, loc: storage)
    net.load_state_dict(parameters)
    return net

net = get_net()

In [ ]:
def total_num_filters(net: nn.Module) -> int:
    n_filters = 0
    for m in net.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            n_filters += m.out_channels
    return n_filters

n_filters = total_num_filters(net)
n_filters_to_prune_per_iter = 512
n_iterations = int(n_filters / n_filters_to_prune_per_iter * 2 / 3)

print('Filters in model:', n_filters)
print('Prune n filters per iteration:', n_filters_to_prune_per_iter)
print('Number of iterations:', n_iterations)